### Step 1: Set up the workspace

In [24]:
import json

#GETTING MY ID:
id = open('id_confidential.json', 'r')
mi = json.load(id)
my_id = mi["azure_id"]

from azureml.core import Workspace

ws = Workspace.create(name="bank",
                      subscription_id = my_id,
                      resource_group = "bank2",
                      location = "centralindia")

from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="model",
                                  workspace=ws)

Deploying KeyVault with name bankkeyvaulte46b5fa6d588.
Deploying StorageAccount with name bankstorage7450f7ff86d64.
Deploying AppInsights with name bankinsightsb1b53a8ac574.
Deployed AppInsights with name bankinsightsb1b53a8ac574. Took 6.4 seconds.
Deployed KeyVault with name bankkeyvaulte46b5fa6d588. Took 25.64 seconds.
Deployed StorageAccount with name bankstorage7450f7ff86d64. Took 27.14 seconds.
Deploying Workspace with name bank.
Deployed Workspace with name bank. Took 41.88 seconds.
Registering model model


In [ ]:
scorepy = """import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('bank_model')
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data'][0]
        data = {key: data[key] for key in [" Persistent EPS in the Last Four Seasons", " Debt ratio %", " Net worth/Assets"]}
        data = pd.DataFrame(data)

        result = model.predict(data)

        return json.dumps({"result": result.tolist()})
    except Exception as e:
        return json.dumps({"error": str(e)})
"""

with open("score.py", "w") as file_score:
    file_score.write(scorepy)

### Step 2: Inference configuration

In [26]:
from azureml.core.environment import Environment
virtual_env = Environment("env_bankrupt")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn'])


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\blast\AppData\Local\Temp\ipykernel_5296\778469531.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [27]:
service.wait_for_deployment()

Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 05f5ed91-fec3-4e5f-8dc7-018c15407324
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 7a963f5a21854f4b9da93e0803c40845.azurecr.io/azureml/azureml_6ade10047ac278b9562561a856ae1185 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for 

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 05f5ed91-fec3-4e5f-8dc7-018c15407324
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 7a963f5a21854f4b9da93e0803c40845.azurecr.io/azureml/azureml_6ade10047ac278b9562561a856ae1185 locally. Please refer to

In [ ]:
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

In [ ]:
print(scoring_uri)

None
